In [ ]:
import sys
print(sys.version)
print(sys.executable)


: 

In [ ]:
import open3d as o3d
import numpy as np

print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("imgs/rabbit.pcd")
print(pcd)

print("->正在保存点云")
o3d.io.write_point_cloud("imgs/write.pcd", pcd)	# 默认false，保存为Binarty；True 保存为ASICC形式
print(pcd)


: 

In [ ]:
import glfw

if not glfw.init():
    print("Failed to initialize GLFW")
else:
    print("GLFW initialized successfully")
    glfw.terminate()


: 

In [9]:
# 可视化
o3d.visualization.draw_geometries([pcd])


[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [4]:
# 可视化多个点云

print("->正在加载点云2...")
pcd1 = o3d.io.read_point_cloud("imgs/bun045.pcd")
print(pcd1)

print("->正在同时可视化两个点云...")
o3d.visualization.draw_geometries([pcd, pcd1])


->正在加载点云2...
PointCloud with 40097 points.
->正在同时可视化两个点云...


In [5]:
# 法线估计
radius = 0.01   # 搜索半径
max_nn = 30     # 邻域内用于估算法线的最大点数
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))     # 执行法线估计

# 可视化
o3d.visualization.draw_geometries([pcd], 
                                  window_name = "可视化参数设置",
                                  width = 600,
                                  height = 450,
                                  left = 30,
                                  top = 30,
                                  point_show_normal = True)


In [6]:
# k-d tree

# 将点云设置为灰色
pcd.paint_uniform_color([0.5, 0.5, 0.5])

# 建立KDTree
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

# 将第1500个点设置为紫色
pcd.colors[1500] = [0.5, 0, 0.5]

# 使用K近邻，将第1500个点最近的5000个点设置为蓝色
print("使用K近邻，将第1500个点最近的5000个点设置为蓝色")
k = 5000    # 设置K的大小
[num_k, idx_k, _] = pcd_tree.search_knn_vector_3d(pcd.points[1500], k)    # 返回邻域点的个数和索引
np.asarray(pcd.colors)[idx_k[1:], :] = [0, 0, 1]  # 跳过最近邻点（查询点本身）进行赋色
print("k邻域内的点数为：", num_k)

# 使用半径R近邻，将第1500个点半径（0.02）范围内的点设置为红色
print("使用半径R近邻，将第15000个点半径（0.02）范围内的点设置为红色")
radius = 0.02   # 设置半径大小
[num_radius, idx_radius, _] = pcd_tree.search_radius_vector_3d(pcd.points[1500], radius)   # 返回邻域点的个数和索引
np.asarray(pcd.colors)[idx_radius[1:], :] = [1, 0, 0]  # 跳过最近邻点（查询点本身）进行赋色
print("半径r邻域内的点数为：", num_radius)

# 使用混合邻域，将半径R邻域内不超过max_num个点设置为绿色
print("使用混合邻域，将第15000个点半径R邻域内不超过max_num个点设置为绿色")
max_nn = 200   # 半径R邻域内最大点数
[num_hybrid, idx_hybrid, _] = pcd_tree.search_hybrid_vector_3d(pcd.points[1500], radius, max_nn)
np.asarray(pcd.colors)[idx_hybrid[1:], :] = [0, 1, 0]  # 跳过最近邻点（查询点本身）进行赋色
print("混合邻域内的点数为：", num_hybrid)

print("->正在可视化点云...")
o3d.visualization.draw_geometries([pcd])
 

使用K近邻，将第1500个点最近的5000个点设置为蓝色
k邻域内的点数为： 5000
使用半径R近邻，将第15000个点半径（0.02）范围内的点设置为红色
半径r邻域内的点数为： 751
使用混合邻域，将第15000个点半径R邻域内不超过max_num个点设置为绿色
混合邻域内的点数为： 200
->正在可视化点云...


In [7]:
# 隐藏点剔除
# 假设希望从给定的视点清染点云，但背号中的点会泄漏到前景中，
# 因为它们不会被其他点遮挡。为此，我们可以应用隐藏点移除算法在Open3D中，
# 实现了[Katz2007]的方法，该方法从给定视图近似点云的可见性，无需曲面重建或法线估计。
print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("imgs/rabbit.pcd")
print(pcd)

print("->正在剔除隐藏点...")

diameter = np.linalg.norm(np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
print("定义隐藏点去除的参数")
camera = [0, 0, diameter]       # 视点位置
radius = diameter * 100         # 噪声点云半径,The radius of the sperical projection
_, pt_map = pcd.hidden_point_removal(camera, radius)    # 获取视点位置能看到的所有点的索引 pt_map

# 可视点点云
pcd_visible = pcd.select_by_index(pt_map)
pcd_visible.paint_uniform_color([0, 0, 1])	# 可视点为蓝色
print("->可视点个数为：", pcd_visible)
# 隐藏点点云
pcd_hidden = pcd.select_by_index(pt_map, invert = True)
pcd_hidden.paint_uniform_color([1, 0, 0])	# 隐藏点为红色
print("->隐藏点个数为：", pcd_hidden)
print("->正在可视化可视点和隐藏点点云...")
#o3d.visualization.draw_geometries([pcd_visible, pcd_hidden])
o3d.visualization.draw_geometries([pcd_visible])


->正在加载点云... 
PointCloud with 35947 points.
->正在剔除隐藏点...
定义隐藏点去除的参数
->可视点个数为： PointCloud with 11490 points.
->隐藏点个数为： PointCloud with 24457 points.
->正在可视化可视点和隐藏点点云...


In [8]:
# 点云曲面重建
# 在许多情况下，我们希望生成密集的三维几何体，即三角形网格。
# 然而，从多视图立体方法或深度传感器中，我们只能获得非结构化点云。
# 要从非结构化输入中获得三角形网格，我们需要执行曲面重建

# ------------------------- Alpha shapes -----------------------
# 这里可以玩一下另外两个数
# alpha = 0.05
# alpha = 0.03
alpha = 0.01
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
# ==============================================================


alpha=0.010


In [9]:
# ------------------------- Ball pivoting --------------------------
print('run Poisson surface reconstruction')
radius = 0.001   # 搜索半径
max_nn = 30         # 邻域内用于估算法线的最大点数
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))     # 执行法线估计

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=12)
print(mesh)
o3d.visualization.draw_geometries([mesh])
# ==============================================================


run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 35947 / 35946
[Open3D DEBUG] #   Got kernel density: 0.458 (s), 495.367 (MB) / 495.367 (MB) / 495 (MB)
[Open3D DEBUG] #     Got normal field: 0.124 (s), 622.52 (MB) / 622.52 (MB) / 622 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 8.608736e-05 / 3.094582e+00
[Open3D DEBUG] #       Finalized tree: 0.506000 (s), 628.953 (MB) / 628.953 (MB) / 761 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.098 (s), 501.348 (MB) / 628.953 (MB) / 761 (MB)
[Open3D DEBUG] #Set point constraints: 0.0900002 (s), 495.23 (MB) / 628.953 (MB) / 761 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 4910270 / 356360 / 5255377
[Open3D DEBUG] Memory Usage: 495.230 MB
[Open3D DEBUG] # Linear system solved: 0.699 (s), 511.215 (MB) / 628.953 (MB) / 761 (MB)
[Open3D DEBUG] Got average: 0.023 (s), 508.488 (MB) / 628.953 (MB) / 761 (MB)
[Open3D DEBUG] Iso-Value: 5.001479e-01 = 1.797882e+04 / 3.594700e+04
[Open3D DEBUG] #          Tota

In [10]:
# 点云空间变换
# 平移
import copy  # 点云深拷贝


print(f'pcd质心：{pcd.get_center()}')
# ===========================================================

# -------------------------- 点云平移 ------------------------
print("\n->沿X轴平移0.2m")
pcd_tx = copy.deepcopy(pcd).translate((0.2, 0, 0))
pcd_tx.paint_uniform_color([1, 0, 0])
print(pcd_tx)
print(f'pcd_tx质心：{pcd_tx.get_center()}')

print("\n->沿Y轴平移0.2m")
pcd_ty = copy.deepcopy(pcd_tx).translate((0, 0.2, 0))
pcd_ty.paint_uniform_color([0, 1, 0])
print(pcd_ty)
print(f'pcd_ty质心：{pcd_ty.get_center()}')

print("\n->沿X轴平移-0.2m，再沿Y轴平移0.2m")
pcd_txy = copy.deepcopy(pcd).translate((-0.2, 0.2, 0))
pcd_txy.paint_uniform_color([0, 0, 1])
print(pcd_txy)
print('pcd_txy质心：', pcd_txy.get_center())
# ===========================================================

# -------------------------- 可视化 --------------------------
#o3d.visualization.draw_geometries([pcd, pcd_tx, pcd_ty, pcd_txy])
o3d.visualization.draw_geometries([pcd, pcd_tx])

# ===========================================================


pcd质心：[-0.02675991  0.09521606  0.00894711]

->沿X轴平移0.2m
PointCloud with 35947 points.
pcd_tx质心：[0.17324009 0.09521606 0.00894711]

->沿Y轴平移0.2m
PointCloud with 35947 points.
pcd_ty质心：[0.17324009 0.29521606 0.00894711]

->沿X轴平移-0.2m，再沿Y轴平移0.2m
PointCloud with 35947 points.
pcd_txy质心： [-0.22675991  0.29521606  0.00894711]


In [11]:
# 将点云质心平移到指定位置

# -------------------------- 点云平移 ------------------------
print("\n->沿X轴平移0.2m")
pcd_tx = copy.deepcopy(pcd).translate((0.2, 0.2, 0.2))
pcd_tx.paint_uniform_color([1, 0, 0])
print(pcd_tx)
print(f'pcd_tx质心：{pcd_tx.get_center()}')

print("\n->将点云质心平移到指定位置")
pcd_new = copy.deepcopy(pcd_tx).translate((0.2, 0.2, 0.2),relative=False)
#pcd_new = copy.deepcopy(pcd_tx).translate((0.2, 0.2, 0.2),False)  # relative 可以省略
pcd_new.paint_uniform_color([0, 1, 0])
print(pcd_new)
print(f'pcd_new：{pcd_new.get_center()}')
# ===========================================================

# -------------------------- 可视化 --------------------------
o3d.visualization.draw_geometries([pcd, pcd_tx, pcd_new])
# ===========================================================



->沿X轴平移0.2m
PointCloud with 35947 points.
pcd_tx质心：[0.17324009 0.29521606 0.20894711]

->将点云质心平移到指定位置
PointCloud with 35947 points.
pcd_new：[0.2 0.2 0.2]


In [12]:
# 旋转
# 未指定旋转中心，默认以点云质心为旋转中心，旋转前后点云质心不变

# -------------------------- 加载点云 ------------------------
pcd.paint_uniform_color([1,0,0])
print("->pcd质心：",pcd.get_center())
# ===========================================================

# -------------------------- 点云旋转 ------------------------
print("\n->采用欧拉角进行点云旋转")
pcd_EulerAngle = copy.deepcopy(pcd)
R1 = pcd.get_rotation_matrix_from_xyz((0,np.pi/2,0))
print("旋转矩阵：\n",R1)
pcd_EulerAngle.rotate(R1)    # 不指定旋转中心
pcd_EulerAngle.paint_uniform_color([0,0,1])
print("\n->pcd_EulerAngle质心：",pcd_EulerAngle.get_center())
# ===========================================================

# -------------------------- 可视化 --------------------------
o3d.visualization.draw_geometries([pcd, pcd_EulerAngle])
# ===========================================================


->pcd质心： [-0.02675991  0.09521606  0.00894711]

->采用欧拉角进行点云旋转
旋转矩阵：
 [[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]

->pcd_EulerAngle质心： [-0.02675991  0.09521606  0.00894711]


In [13]:
# 指定旋转中心，旋转前后点云质心改变
# -------------------------- 加载点云 ------------------------
print(pcd)
pcd.paint_uniform_color([1,0,0])
print("->pcd质心：",pcd.get_center())
# ===========================================================

# -------------------------- 点云旋转 ------------------------
print("\n->采用欧拉角进行点云旋转")
pcd_EulerAngle = copy.deepcopy(pcd)
R1 = pcd.get_rotation_matrix_from_xyz((0,np.pi/2,0))
print("旋转矩阵：\n",R1)
pcd_EulerAngle.rotate(R1,center = (0.1,0.1,0.1))    # 指定旋转中心
pcd_EulerAngle.paint_uniform_color([0,0,1])
print("\n->pcd_EulerAngle质心：",pcd_EulerAngle.get_center())
# ===========================================================

# -------------------------- 可视化 --------------------------
o3d.visualization.draw_geometries([pcd, pcd_EulerAngle])
# ===========================================================


PointCloud with 35947 points.
->pcd质心： [-0.02675991  0.09521606  0.00894711]

->采用欧拉角进行点云旋转
旋转矩阵：
 [[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]

->pcd_EulerAngle质心： [0.00894711 0.09521606 0.22675991]


In [14]:
# 计算点云最小包围盒
print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("imgs/rabbit.pcd")
print(pcd)

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
print("->正在计算点云轴向最小包围盒...")
aabb = pcd.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
print("->正在计算点云最小包围盒...")
obb = pcd.get_oriented_bounding_box()
obb.color = (0, 1, 0)
o3d.visualization.draw_geometries([pcd, aabb, obb, coordinate_frame])


->正在加载点云... 
PointCloud with 35947 points.
->正在计算点云轴向最小包围盒...
->正在计算点云最小包围盒...


In [15]:
# 计算点云凸包
# 点云的凸包是包含所有点的最小凸集。0pen3D 包含计算点云凸包的 compute_convex_hu11 方法。该实现基于Qhul。

print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("imgs/rabbit.pcd")
print(pcd)

print("->正在计算点云凸包...")
hull, _ = pcd.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([pcd, hull_ls])


->正在加载点云... 
PointCloud with 35947 points.
->正在计算点云凸包...


In [16]:
# 点云体素化

# --------------------------- 体素化点云 -------------------------
print('执行体素化点云')
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=0.005)
print("正在可视化体素...")
o3d.visualization.draw_geometries([voxel_grid])
# ==============================================================


执行体素化点云
正在可视化体素...


In [17]:
pcd_new = copy.deepcopy(pcd_tx).translate((0.2, 0.2, 0.2),relative=False)# 点云赋色

print("->正在点云赋色...")
pcd.paint_uniform_color([1,0.706,0])
print("->正在可视化赋色后的点云...")
o3d.visualization.draw_geometries([pcd])

print("->正在保存赋色后的点云")
o3d.io.write_point_cloud("imgs/color.pcd", pcd)	# 默认false，保存为Binarty；True 保存为ASICC形式


->正在点云赋色...
->正在可视化赋色后的点云...
->正在保存赋色后的点云


True